In [ ]:
import pandas as pd
import numpy as np

# 절대 모멘텀
1. 주식 데이터 로드
2. 년월 파생변수 생성 -> Date 컬럼에서 년-월 추출하여 대입
3. 년, 월 별 마지막날의 month_last_dt 데이터프레임을 생성
4. 전월의 종가를 가지는 파생변수 하나 생성
5. 전년도의 종가를 가지는 파생변수 하나 생성
6. 전월, 전년도의 종가를 가지고 거래내역 생성
7. 수익율 계산

In [ ]:
df = pd.read_csv("../csv/GDX.csv", index_col = "Date")
df.head(3)

In [ ]:
# index를 시계열 데이터로 변경
df.index = pd.to_datetime(df.index, format = "%Y-%m-%d")

# to_datetime()의 기능을 구현

In [ ]:
from datetime import datetime

In [ ]:
# datetime 라이브러리를 이용하여 시계열로 변경
test_df = df.copy()

In [ ]:
# 리셋 인덱스
# case 1
test_df = test_df.reset_index()
test_df["Date"] = test_df["Date"].apply(lambda x : datetime.strptime(x, "%Y-%m-%d"))
test_df.set_index("Date")

In [ ]:
# case 2
test_df.index

def change(x) :
    return datetime.strptime(x, "%Y-%m-%d")

test_df.index = list(map(change, test_df.index))     

In [ ]:
test_df.head(2)

In [ ]:
# map 함수
x = [1, 2, 3, 4]
def change(x) :
    return x ** 2

list(map(lambda y : y ** 2, x))

# 모멘텀 어게인

In [ ]:
# 새로운 파생변수 년-월 데이터를 대입
df["STD-YM"] = list(map(lambda x : datetime.strftime(x, "%Y-%m"), df.index))
df.head()

In [ ]:
# 월별 마지막 인덱스만 추출하여 새로운 데이터프레임에 대입

# case 1 (shift 이용)
# 현재의 STM-YM의 값과 다음 행의 STD-YM의 값이 다른 경우
df.loc[df["STD-YM"] != df.shift(-1)["STD-YM"]]


In [ ]:
# case 2 (for 이용)
month_list_df = pd.DataFrame()
_list = df["STD-YM"].unique()

for i in _list :
    last_df = df.loc[df["STD-YM"] == i].tail(1)
    month_list_df = pd.concat([month_list_df, last_df])

month_list_df

In [ ]:
# 전월의 종가(Adj Close)를 가지는 파생변수(BF_1M)을 생성하고 결측치는 0으로 대체
# 전년도의 종가(Adj Close)를 가지는 파생변수(BF_12M)을 생성하고 결측치는 0으로 대체
month_list_df["BF_1M"] = month_list_df.shift(1)["Adj Close"].fillna(0)
month_list_df["BF_12M"] = month_list_df.shift(12)["Adj Close"].fillna(0)
month_list_df.head(13)

In [ ]:
# 거래내역 추가
# 구매 조건 -> ((전월 종가 / 전 년도 종가) - 1)의 값이 0보다 크고 무한대가 아닌 경우

df["trade"] = ""

for i in month_list_df.index :
    signal = ""

    # 절대 모멘텀 계산
    momentum_index = month_list_df.loc[i, "BF_1M"] / month_list_df.loc[i, "BF_12M"] - 1

    # 절대 모멘텀 지표에 따라서 True 와 False 로 구분
    flag = True if((momentum_index > 0) and (momentum_index != -np.inf) and(momentum_index != np.inf)) else False

    if flag :
        signal = "buy"

    print("날짜 : ", i, "모멘텀 인덱스 : ", momentum_index, "flag : ", flag, "signal : ", signal)

    df.loc[i, "trade"] = signal

In [ ]:
df["trade"].value_counts()

In [ ]:
df.head()

### 수익율 계산
1. 파생변수(return) 값은 1로 생성
2. rtn = 1.0, buy, sell 변수는 0으로 생성
3. 반복문을 이용해서 현재의 trade가 buy이고 전 행의 trade가 "" 인 경우 구매 날
4. 전 행의 trade가 buy이고 현재의 trade가 "" 인 경우 판매 날
5. 수익율 계산해서 return 대입

### 누적 수익율 계산
1. acc_rtn = 1.0 생성
2. return의 항목들을 누적 곱하여 acc_rtn 컬럼에 대입
3. 최종적으로 acc_rtn을 출력

In [ ]:
# 1. 파생변수(return) 값은 1로 생성
df["return"] = 1
# 2. rtn = 1.0, buy, sell 변수는 0으로 생성
rtn = 1.0 
buy = 0
sell = 0

for i in df.index :
    # 3. 반복문을 이용해서 현재의 trade가 buy이고 전 행의 trade가 "" 인 경우 구매 날
    if (df.loc[i, "trade"] == "buy") and (df.shift(1).loc[i, "trade"] == "") :
        buy = df.loc[i, "Adj Close"]
        print("진입일 : ", i, "구입 가격 : ", buy)
    # 4. 전 행의 trade가 buy이고 현재의 trade가 "" 인 경우 판매 날
    elif (df.shift(1).loc[i, "trade"] == "buy") and (df.loc[i, "trade"] == "") :
        sell = df.loc[i, "Adj Close"]
        # 5. 수익율 계산해서 return 대입
        rtn = (sell - buy) / buy + 1
        df.loc[i, "return"] = rtn
        print("판매일 : ", i, "판매가격 : ", sell, "수익률 : ", rtn)



In [ ]:
acc_rtn = 1.0

for i in df.index :
    acc_rtn *= df.loc[i, "return"]
    df.loc[i, "acc_rtn"] = acc_rtn
print("누적수익율 : ", acc_rtn)

In [ ]:
df["return"].cumprod()

# 절대 모멘텀을 함수화
1. 첫번째 함수
    - 매개변수 1게(데이터프레임)
    - 데이터프레임에 결측치와 이상치를 제거
    - 수정종가를 제외한 나머지 컬럼은 삭제
    - 인덱스를 시계열로 변경
    - "STD-YM" 파생변수를 생성하여 인덱스의 년-월 추출해서 대입
2. 두번째 함수
    - 매개변수 1개(데이터프레임)
    - 새로운 데이터프레임 생성
    - 인자값으로 받아온 데이터프레임에서 년-월별 마지막 데이터들을 새로운 데이터프레임에 대입
    - df에 파생변수 2개 생성
        - "BF_1M" : 전월의 종가, 결측치는 0으로 대체
        - "BF_12M" : 전년도의 종가, 결측치는 0으로 대체
    - df2를 리턴
3. 세번째 함수
    - 매개변수 2개(첫번째 함수의 결과(df1), 두번째 함수의 결과(df2))
    - df1에 trade 컬럼을 생성, 값은 ""
    - df1에 return 컬럼을 생성, 값은 1
    - df2의 값들을 이용하여 momentum_index를 구하고 df1에 거래 내역 삽입
    - df1의 거래 내역을 이용하여 수익율 return 컬럼에 대입
    - return 컬럼의 데이터를 가지고 누적수익율(acc_rtn)에 대입
    - 누적수익율을 print를 이용하여 출력
    - df1을 리턴

In [81]:
import pandas as pd
import numpy as np
from datetime import datetime

# 1. 첫번째 함수
# 매개변수 1게(데이터프레임)
def mmt1 (df) :
    # 인덱스가 Date가 아니면? Date 컬럼을 인덱스로 변경
    if "Date" in df.columns :
        df = df.set_index("Date")
    # 데이터프레임에 결측치와 이상치를 제거, 수정종가를 제외한 나머지 컬럼은 삭제
    df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(1), ["Adj Close"]]
    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index, format = "%Y-%m-%d")
    # "STD-YM" 파생변수를 생성하여 인덱스의 년-월 추출해서 대입
    df["STD-YM"] = list(map(lambda x : datetime.strftime(x, "%Y-%m"), df.index))
    return df

In [84]:
data = pd.read_csv("../csv/GDX.csv")
df1 = mmt1(data)

C:\Users\duja1\AppData\Local\Temp\ipykernel_19928\2805242224.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(1), ["Adj Close"]]


In [85]:
# 2. 두번째 함수
# 매개변수 1개(데이터프레임)
def mmt2 (df) :
    # 새로운 데이터프레임 생성
    df2 = pd.DataFrame()
    # 인자값으로 받아온 데이터프레임에서 년-월별 마지막 데이터들을 새로운 데이터프레임에 대입
    df2 = df.loc[df["STD-YM"] != df.shift(-1)["STD-YM"]]
    # df에 파생변수 2개 생성
    # "BF_1M" : 전월의 종가, 결측치는 0으로 대체
    df2["BF_1M"] = df2.shift(1)["Adj Close"].fillna(0)
    # "BF_12M" : 전년도의 종가, 결측치는 0으로 대체
    df2["BF_12M"] = df2.shift(12)["Adj Close"].fillna(0)
    return df2
    

In [87]:
df2 = mmt2(df1)

C:\Users\duja1\AppData\Local\Temp\ipykernel_19928\2207663293.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_list_df["BF_1M"] = month_list_df.shift(1)["Adj Close"].fillna(0)
C:\Users\duja1\AppData\Local\Temp\ipykernel_19928\2207663293.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_list_df["BF_12M"] = month_list_df.shift(12)["Adj Close"].fillna(0)


In [97]:
# 3. 세번째 함수
# 매개변수 2개(첫번째 함수의 결과(df1), 두번째 함수의 결과(df2))
def mmt3(df1, df2) :
    # df1에 trade 컬럼을 생성, 값은 ""
    df1["trade"] = ""
    # df1에 return 컬럼을 생성, 값은 1
    df1["return"] = 1
    # df2의 값들을 이용하여 momentum_index를 구하고 df1에 거래 내역 삽입
    for i in df2.index :
        signal = ""
        # 절대 모멘텀 계산
        momentum_index = df2.loc[i, "BF_1M"] / df2.loc[i, "BF_12M"] - 1
        # 절대 모멘텀 지표에 따라서 True 와 False 로 구분
        flag = True if((momentum_index > 0) and (momentum_index != -np.inf) and(momentum_index != np.inf)) else False
        if flag :
            signal = "buy"
        df1.loc[i, "trade"] = signal
    # df1의 거래 내역을 이용하여 수익율 return 컬럼에 대입
    # 2. rtn = 1.0, buy, sell 변수는 0으로 생성
    rtn = 1.0 
    buy = 0
    sell = 0

    for i in df1.index :
        # 3. 반복문을 이용해서 현재의 trade가 buy이고 전 행의 trade가 "" 인 경우 구매 날
        if (df1.loc[i, "trade"] == "buy") and (df1.shift(1).loc[i, "trade"] == "") :
            buy = df1.loc[i, "Adj Close"]
            print("구입일 : ", i, "구입 가격 : ", buy)
        # 4. 전 행의 trade가 buy이고 현재의 trade가 "" 인 경우 판매 날
        elif (df1.shift(1).loc[i, "trade"] == "buy") and (df1.loc[i, "trade"] == "") :
            sell = df1.loc[i, "Adj Close"]
            # 5. 수익율 계산해서 return 대입
            print("판매일 : ", i, "판매가격 : ", sell)
            rtn = (sell - buy) / buy + 1
            df1.loc[i, "return"] = rtn
            
    # return 컬럼의 데이터를 가지고 누적수익율(acc_rtn)에 대입
    acc_rtn = 1.0
    for i in df1.index :
        acc_rtn *= df1.loc[i, "return"]
        df1.loc[i, "acc_rtn"] = acc_rtn
    print(acc_rtn)

    return df1

In [98]:
mmt3(df1, df2)

C:\Users\duja1\AppData\Local\Temp\ipykernel_19928\3850679863.py:12: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = df2.loc[i, "BF_1M"] / df2.loc[i, "BF_12M"] - 1
C:\Users\duja1\AppData\Local\Temp\ipykernel_19928\3850679863.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = df2.loc[i, "BF_1M"] / df2.loc[i, "BF_12M"] - 1


구입일 :  2007-05-31 00:00:00 구입 가격 :  36.162109
판매일 :  2007-06-01 00:00:00 판매가격 :  36.993214
구입일 :  2007-06-29 00:00:00 구입 가격 :  34.989338
판매일 :  2007-07-02 00:00:00 판매가격 :  35.866615
구입일 :  2007-09-28 00:00:00 구입 가격 :  41.878242
판매일 :  2007-10-01 00:00:00 판매가격 :  42.312256
구입일 :  2007-10-31 00:00:00 구입 가격 :  46.72633
판매일 :  2007-11-01 00:00:00 판매가격 :  45.082588
구입일 :  2007-11-30 00:00:00 구입 가격 :  42.810921
판매일 :  2007-12-03 00:00:00 판매가격 :  43.235706
구입일 :  2007-12-31 00:00:00 구입 가격 :  43.033947
판매일 :  2008-01-02 00:00:00 판매가격 :  46.235912
구입일 :  2008-01-31 00:00:00 구입 가격 :  47.278187
판매일 :  2008-02-01 00:00:00 판매가격 :  46.151402
구입일 :  2008-02-29 00:00:00 구입 가격 :  49.879185
판매일 :  2008-03-03 00:00:00 판매가격 :  51.287678
구입일 :  2008-03-31 00:00:00 구입 가격 :  44.78986
판매일 :  2008-04-01 00:00:00 판매가격 :  43.559784
구입일 :  2008-04-30 00:00:00 구입 가격 :  40.939995
판매일 :  2008-05-01 00:00:00 판매가격 :  40.047958
구입일 :  2008-05-30 00:00:00 구입 가격 :  43.212353
판매일 :  2008-06-02 00:00:00 판매가격 :  43.57856
구입

,Adj Close,STD-YM,trade,return,acc_rtn
Date,,,,,
2006-05-22,34.280224,2006-05,,1.0,1.000000
2006-05-23,34.952393,2006-05,,1.0,1.000000
2006-05-24,33.626480,2006-05,,1.0,1.000000
2006-05-25,35.283867,2006-05,,1.0,1.000000
2006-05-26,35.495640,2006-05,,1.0,1.000000
...,...,...,...,...,...
2019-06-18,23.670000,2019-06,,1.0,1.165002
2019-06-19,24.000000,2019-06,,1.0,1.165002
2019-06-20,25.049999,2019-06,,1.0,1.165002
